## Создание массива слов
Для того, чтобы находить и исправлять опечатки с помощью расстояния Левенштейна, сперва необходимо создать массив слов, чтобы мы могли найти правильно написанный вариант слова, в котором допустили опечатку.

Для этого мы извлёчем все слова из корпуса. Корпус это массив языковых данных, например письменных текстов, аудиозаписей и так далее. Корпуса представлены в электронном виде для решения разных задач. Например, для решения нашей задачи.

### Загрузка корпуса
Мы воспользуемся одним из немногих доступных корпусов английского языка — Brown Corpus. Для начала необходимо добавить корпус в google colab. Следуй инструкции:
1. Открываем files google colab (папка в панель слева).
2. Нажимаем левой кнопкой мыши по папке с названием "...".
3. Нажимаем "новая папка"/"new folder".
4. В качестве названия указываем Brown.
5. Кликаем левой кнопкой мыши по папке Brown.
6. Нажимаем "загрузить"/"upload".
7. Находим корневую папку проекта.
8. Открываем в корневой папке проекта папку Brown.
9. Выделяем все файлы комбинацией клавиш "ctrl + A".
10. Нажимаем "открыть"/"open".

После того, как все файлы прогрузятся (около минуты) перейдём к следующей задаче.

### Обработка корпуса
После того, как мы загрузили корпус, необходимо представить все эти текстовые файлы в удобном для нас формате. Проще всего будет воспользоваться функционалом библиотеки corpus_toolkit. Запусти код ниже, чтобы установить всё необходимое:

In [2]:
!pip install corpus-toolkit  # Устанавливаем библиотеку в google colab

from corpus_toolkit import corpus_tools as ct  # Импортируем библиотеку
import spacy  # Импортируем вспомогательную библиотеку

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.0 MB/s eta 0:00:00


Эта библиотека создана для создания так называемых словарей частотностей. Это такие пары ключ-значение, где ключ это слово, а значение количество появлений в корпусе. Мы создадим словарь частотности, а затем вытащим из него все ключи, воспользовавшись кодом из [документации](https://github.com/kristopherkyle/corpus_toolkit)

In [4]:
brown_corp = ct.ldcorpus('Brown')  # Загружаем корпус
tok_corp = ct.tokenize(brown_corp)  # Токенизируем корпус
brown_freq = ct.frequency(tok_corp)  # Создаём словарь частотности

Processing ch_ch06.txt (1 of 500 files)
Processing cg_cg27.txt (2 of 500 files)
Processing cf_cf20.txt (3 of 500 files)
Processing ck_ck01.txt (4 of 500 files)
Processing ca_ca04.txt (5 of 500 files)
Processing ck_ck18.txt (6 of 500 files)
Processing cd_cd12.txt (7 of 500 files)
Processing ca_ca07.txt (8 of 500 files)
Processing cp_cp01.txt (9 of 500 files)
Processing cf_cf04.txt (10 of 500 files)
Processing cb_cb09.txt (11 of 500 files)
Processing cj_cj07.txt (12 of 500 files)
Processing cg_cg08.txt (13 of 500 files)
Processing cd_cd15.txt (14 of 500 files)
Processing cj_cj61.txt (15 of 500 files)
Processing cr_cr02.txt (16 of 500 files)
Processing cg_cg60.txt (17 of 500 files)
Processing cj_cj18.txt (18 of 500 files)
Processing cl_cl12.txt (19 of 500 files)
Processing cb_cb22.txt (20 of 500 files)
Processing cd_cd10.txt (21 of 500 files)
Processing ca_ca42.txt (22 of 500 files)
Processing cc_cc06.txt (23 of 500 files)
Processing cf_cf39.txt (24 of 500 files)
Processing cg_cg17.txt (2

Давайте посмотрим действительно ли всё сработало, выведем самые частотные слова корпуса:

In [7]:
ct.head(brown_freq, hits = 5)  # Вывод 5 самых частотных слов

the	69836
be	37689
of	36365
a	30475
and	28826


Отлично, теперь постараемся разобраться с тем, что такое словарь и как извлечь из него ключи. Словарь это тип данных на python, который позволяет хранить пары ключ-значение, например:

In [9]:
# Словарь иницилизируется с помощью фигурных скобок {}
grammy_awards_dict = {
    'Kanye West': 24,  # Внутри словаря мы пишем ключ, затем двоеточие,
                       # а затем значение
    'Jay-Z': 24,
    'Beyoncé': 32
}

# Выведем словарь, чтобы посмотреть, что у нас получилось
print(grammy_awards_dict)

{'Kanye West': 24, 'Jay-Z': 24, 'Beyoncé': 32}


Нам будет удобней, если словарь частотности будет выстроен в порядке убывания, чтобы мы начинали поиск правильного написания слова с самых частотных слов. Посмотри на код ниже:

In [15]:
sorted_grammy_awards_dict = sorted(
    grammy_awards_dict.items(),  # Список, который мы сортируем
    key=lambda items: items[1],   # Ключ сортировки
    reverse=True  # Порядок сортировки
    )
print(sorted_grammy_awards_dict)  # Список с отсортированными парами ключ-значение
print(dict(sorted_grammy_awards_dict))  # Словарь из этого списка

[('Beyoncé', 32), ('Kanye West', 24), ('Jay-Z', 24)]
{'Beyoncé': 32, 'Kanye West': 24, 'Jay-Z': 24}


С помощью `sorted()` мы создаём список значений, для которого мы можем указать принцип сортировки. В качестве основы мы берём список пар ключ-значение (grammy_awards_dict.items()), он выглядит вот так `dict_items([('Kanye West', 24), ('Jay-Z', 24), ('Beyoncé', 32)])`. В качестве ключа сортировки мы указываем функцию лямбда. Не будем углубляться в подробности, она просто выбирает из пары ключ-значение (items) значение частотности (items[1]). Почему 1, а не 2? Нумерация начинается с 0. В конце мы просто указываем порядок сортировки: `reverse=True`.

Таким образом мы получаем список из пар ключ-значение, которые сортированы в порядке убывания по частоте. Если мы положим список внутрь функции dict(), то получим словарь.

Теперь повтори это, но используй словарь brown_freq:

In [ ]:
# Создай список с сортированными парами ключ-значение
sorted_brown_freq =

# Сделай из списка пар словарь с помощью функции dict().
# Назови его brown_freq
brown_freq =

In [17]:
# Создай список с сортированными парами ключ-значение
sorted_brown_freq = sorted(
    brown_freq.items(),
    key=lambda items: items[1],
    reverse=True
)

# Сделай из списка пар словарь с помощью функции dict().
# Назови его brown_freq
brown_freq = dict(sorted_brown_freq)

In [18]:
# Запусти этот код, чтобы проверить, всё ли получилось
print(brown_freq)

{'the': 69836, 'be': 37689, 'of': 36365, 'a': 30475, 'and': 28826, 'to': 26126, 'in': 21318, 'he': 19417, 'have': 11938, 'it': 10932, 'that': 10777, 'for': 9479, 'i': 8409, 'they': 8217, 'with': 7270, 'on': 6729, 'she': 6022, 's': 5927, 'at': 5368, 'by': 5299, 'this': 5138, 'we': 4996, 'not': 4860, 'you': 4567, 'do': 4411, 'but': 4381, 'from': 4366, 'or': 4186, 'an': 3746, 'which': 3554, 'one': 3475, 'all': 2995, 'would': 2843, 'there': 2839, 'say': 2778, 'will': 2405, 'make': 2345, 'when': 2324, 'who': 2280, 'more': 2230, 'can': 2209, 'no': 2202, 'if': 2193, 'man': 2134, 'out': 2095, 'nt': 2085, 'other': 2029, 'so': 1983, 'what': 1963, 'time': 1921, 'up': 1892, 'go': 1815, 'about': 1815, 'than': 1790, 'into': 1790, 'could': 1776, 'only': 1744, 'new': 1668, 'year': 1658, 'some': 1617, 'take': 1577, 'these': 1573, 'good': 1566, 'come': 1561, 'state': 1558, 'see': 1524, 'get': 1477, 'know': 1474, 'may': 1415, 'two': 1411, 'use': 1400, 'like': 1380, 'then': 1377, 'first': 1360, 'any': 134

Молодец, ты справился. Осталось самое простое. Мы делали из словаря пары ключ-значение с помощью метода `items()`, когда хотели его отсортировать. Вот так: `brown_freq.items()`. Аналогичным образом мы можем получить только ключи, используя метод `keys()`.

Создай новую переменную `brown_freq_keys` и присвой ей ключи нового словаря:

In [ ]:
# Присвой переменной brown_freq_keys ключи словаря brown_freq
brown_freq_keys =

In [19]:
# Присвой переменной brown_freq_keys ключи словаря brown_freq
brown_freq_keys = brown_freq.keys()

In [20]:
# Запусти этот код, чтобы проверить, всё ли получилось
print(brown_freq_keys)

dict_keys(['the', 'be', 'of', 'a', 'and', 'to', 'in', 'he', 'have', 'it', 'that', 'for', 'i', 'they', 'with', 'on', 'she', 's', 'at', 'by', 'this', 'we', 'not', 'you', 'do', 'but', 'from', 'or', 'an', 'which', 'one', 'all', 'would', 'there', 'say', 'will', 'make', 'when', 'who', 'more', 'can', 'no', 'if', 'man', 'out', 'nt', 'other', 'so', 'what', 'time', 'up', 'go', 'about', 'than', 'into', 'could', 'only', 'new', 'year', 'some', 'take', 'these', 'good', 'come', 'state', 'see', 'get', 'know', 'may', 'two', 'use', 'like', 'then', 'first', 'any', 'now', 'such', 'even', 'give', 'over', 'work', 'most', 'think', 'afj', 'day', 'after', 'also', 'find', 'way', 'many', 'must', 'look', 'before', 'back', 'long', 'through', 'where', 'great', 'much', 'well', 'down', 'people', 'should', 'because', 'each', 'just', 'those', 'mr', 'how', 'seem', 'too', 'little', 'world', 'own', 'place', 'leave', 'life', 'very', 'feel', 'still', 'old', 'tell', 'become', 'here', 'hand', 'show', 'house', 'mean', 'both', 

Супер! Осталось исправить одну деталь. Ключи из словаря находятся в формате dict_keys, а нам нужен формат list. Запусти код ниже, чтобы поменять формат:

In [21]:
print(type(brown_freq_keys))  # Тип до изменений

brown_freq_keys = list(brown_freq_keys)

print(type(brown_freq_keys))  # Тип после изменений

<class 'dict_keys'>
<class 'list'>


Отлично, у тебя всё получилось! В следующем разделе мы сохраним этот список в удобном формате.

### Сохранение списка слов в виде json массива
Если мы каждый раз будем загружать весь корпус для того, чтобы извлечь список, у нас будет уходить на это несколько секунд. Кажется, что это немного, но кто станет ждать 5 секунд, чтобы увидеть свои опечатки? Мы воспользуемся одним из самых удобных форматов хранения данных — json.

JSON (JavaScript Object Notation) — текстовый формат обмена данными. Бывают json объекты, массивы, строки, числа и литералы. Нас будет интересовать json массив, то есть просто список элементов. В качестве элементов будут наши слова.

Но для начала разберёмся с тем, как на python работать с файлами. Всё, что нужно сейчас это простая конструкция ниже:

In [ ]:
with open('Brown/ca_ca01.txt', 'r') as file:
  print(file.read())

The Fulton County Grand Jury said Friday an investigation of Atlanta 's recent primary election produced " no evidence " that any irregularities took place . 
The jury further said in term-end presentments that the City Executive Committee , which had over-all charge of the election , " deserves the praise and thanks of the City of Atlanta " for the manner in which the election was conducted . 
The September-October term jury had been charged by Fulton Superior Court Judge Durwood Pye to investigate reports of possible " irregularities " in the hard-fought primary which was won by Mayor-nominate Ivan Allen Jr .. " Only a relative handful of such reports was received " , the jury said , " considering the widespread interest in the election , the number of voters and the size of this city " . 
The jury said it did find that many of Georgia 's registration and election laws " are outmoded or inadequate and often ambiguous " . 
It recommended that Fulton legislators act " to have these law

Мы открыли файл из корпуса, указав в аргументе функции open() путь к файлу в кавычках и способ взаимодействия: `'r'` это чтение (read), `'w'` это чтение и ввод (write). Если мы укажем `'w'` вместо `'r'`, то при отсутствии файла, он будет автоматически создан. Поменяй способ взаимодействия и запусти код. В files google colab должен появиться новый файл:

In [ ]:
with open('new_file.json', 'r') as f:  # Поменяй r на w
  pass  # pass значит отсутствие действия

In [ ]:
with open('new_file.json', 'w') as f:  # Поменяй r на w
  pass  # pass значит отсутствие действия

Теперь импортируем библиотеку json и воспользуемся методом dump. В files google colab должен появиться массив после выполнения следующего кода:

In [ ]:
import json

with open('words_massive.json', 'w') as f:
  json.dump(brown_freq_keys, f)  # В качестве аргумента функции указываем то,
                                 # что мы хотим сохранить, то есть список слов
                                 # и то, куда мы хотим сохранить, то есть в
                                 # созданный нам файл f

Вы можете открыть его двойным щелчком и посмотреть, что там действительно внутри все слова из нашего корпуса. Однако нам нужно будет использовать этот массив на python, а поэтому придётся загрузить json файл в переменную с помощью библиотеки. Для этого напиши простой код:

1. Открой файл words_massive для чтения.
3. Присвой переменной words_massive значение функции `json.load()`, указав в аргументе файл.

In [ ]:
with open('words_massive.json', 'r') as f:
  words_massive = json.load(f)

In [ ]:
# Запусти этот код, чтобы проверить сколько слов в нашем списке

print(len(words_massive))

36377


### Создание функции
Только что ты создал в отдельных ячейках кода создание списка слов из корпуса. Но нам будет удобнее поместить это в функцию, которую мы сможем легко перемещать и использовать. Что такое функции и с чем их едят? Разберёмся на примере:

In [ ]:
def sum_numbers(a, b):
  return a + b

print(sum_numbers(5, 10))

15


Вот наглядный пример функции, которая складывает 2 числа, которые передаются ей в качестве аргумента, то есть те числа, которые мы пишем внутри круглых скобок функции:

```sum_numbers(5, 10)  # 5 и 10 это числа, они же аргумент функции```

Напиши свою функцию, которая будет отнимать числа:

In [ ]:
def difference_in_numbers(a, b):
  return a - b

print(difference_in_numbers(5, 10))

-5


Скорее всего у тебя получилось повторить пример. Давай разберём, как ты это сделал:

1. `def` это такое кодовое слово, с которого мы начинаем нашу функцию.
2. После `def` идёт название функции. Оно может быть каким угодно, но лучше не повторять названий уже существующих функций, иначе можно натворить дел.
3. После названия в круглых скобках указываем аргумент. Его может вовсе не быть, он может быть необязательным (значения указаны по умолчанию) или он может быть обязательным, как в нашем примере.
4. Заканчивается всё двоеточием, после которого у нас начинается "территория" нашей функции, в которую входят строки, которые начинаются с нескольких пробелов от начала и заканчивается теми строками, которые находятся на уровне def:
```
def blah_blah():
      # территория функции
      # территория функции
      # территория функции
# территория вне функции
```
5. В "теле" функции мы определяем действия, которые будет совершать функция при вызове. В нашем примере это return, то есть мы получим обратно какое-то значение, в нашем случае разность двух чисел.

Чтобы проверить, как ты усвоил информацию, создай функцию
`get_words_massive()`, которая будет открывать файл words_massive.json и читать массив слов в переменную. Переменную нужно будет вернуть с помощью return

In [ ]:
def get_words_massive():
  # Открой words_massive.json и с помощью json загрузи массив в переменную


  # Верни переменную с помощью return

In [ ]:
def get_words_massive():
  # Открой words_massive.json и с помощью json загрузи массив в переменную
  with open('words_massive.json', 'r') as f:
    words = json.load(f)

  # Верни переменную с помощью return
  return words

In [ ]:
# Запусти этот код, чтобы проверить сколько слов в нашем массиве
print(len(get_words_massive()))

36377


Поздравляю, ты отлично справился/справилась :)
Теперь, пожалуйста, скачай файл words_massive.json в корневую папку проекта, нажав по нему левой кнопкой мыши, а затем "скачать"/"download"